In [22]:
import sqlite3
import pandas as pd
DATA_ROOT="/home/mcn26/palmer_scratch/tabula_data/raw_recap/cohen_retina/counts"
DB_NAME=f"{DATA_ROOT}/cohen_munging.db"
umi_len=12
pBC_len=8

In [19]:
def break_up_mpra_count(file_name_in,table_name):
    #processes mBC data and dumps it in a sqlite database for rapid access & subsequent processing. 
    #format for MPRA barcodes is :
    #leading whitespace, num occurances, space, 16 bp cell-barcode, underscore, 12 bp umi, 8 bp pBC, 24 bp rBC
    #see readme for derivation

    #establish conection to sql database
    conn=sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    #create table
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        num_occurrences INTEGER,
        cell_barcode TEXT,
        umi TEXT,
        pBC TEXT,
        rBC TEXT
    )
    """)
    

    with open(f"{DATA_ROOT}/{file_name_in}") as fin:
        for line in fin:
            line=line.strip()
            
            #break off number of occurrences
            line=line.split(" ")
            assert len(line) == 2, "err, multiple spaces"
            num_occurrences=int(line[0])

            #break off cell barcode
            line=line[1].split("_")
            assert len(line) == 2, "err, multiple underscore"
            cell_barcode=line[0]
            assert len(cell_barcode)==16, "err, malformed cell barcode"

            #break out remaining barcodes
            line=line[1]
            assert len(line)==44, "err, malformed remaining barcode"
            

            umi=line[0:umi_len]
            pBC=line[umi_len:umi_len+pBC_len]
            rBC=line[umi_len+pBC_len:]

            cursor.execute(f"""
                INSERT INTO {table_name} (num_occurrences, cell_barcode, umi, pBC, rBC)
                VALUES (?, ?, ?, ?, ?)
            """, (num_occurrences, cell_barcode, umi, pBC, rBC))

    
    conn.commit()
    conn.close()


break_up_mpra_count("retina_mpra_r1.count","retina_mpra_r1")

In [20]:
#check to make sure table looks OK.
conn=sqlite3.connect(DB_NAME)
cursor = conn.cursor()
query="""
SELECT *
FROM retina_mpra_r1
LIMIT 5;
"""
c=pd.read_sql_query(query,conn)
conn.close()
c


,id,num_occurrences,cell_barcode,umi,pBC,rBC
0,1,1,AAAAAACAGCGCATCC,CATGCTTATCCC,CACGGTAG,ATCCTACTAACGCATTGTGAATTG
1,2,2,AAAAAACAGCGCATCC,TATCTTAATTGG,CGGAAGGA,TTAATATGCTTAGTTTTTTCATGA
2,3,2,AAAAAACAGGTAAGTT,TCCCACAATCTT,ATAAGAGG,CTCTCGTTCGATATGGCATACTTG
3,4,1,AAAAAACGTTTACGAC,GCTCTCAGGAGA,CGGAAGGA,GGAAGAACCCATTCCTTGCTTTTA
4,5,1,AAAAAACGTTTACGAC,GTTTGTTGATAT,CGAATATC,AGAATCCGGAGAGGCCTTGCAGTA


In [23]:
def break_up_u6_count(filename,table_name):
    #format is
    #leading whitespace, num occurances, space, 16 bp cell-barcode, 12bp UMI, 8 bp pBC
    
    #establish conection to sql database
    conn=sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    #create table
    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        num_occurrences INTEGER,
        cell_barcode TEXT,
        umi TEXT,
        pBC TEXT,
        rBC TEXT
    )
    """)

    with open(f"{DATA_ROOT}/{filename}") as fin:
        for line in fin:
            original_line=line

            line=line.strip()
            
            #break off number of occurrences
            line=line.split(" ")
            assert len(line) == 2, "err, multiple spaces"
            num_occurrences=int(line[0])

            #break off cell barcode
            line=line[1].split("_")
            assert len(line) == 2, "err, multiple underscore"
            cell_barcode=line[0]
            assert len(cell_barcode)==16, "err, malformed cell barcode"

            #break out remaining barcodes
            line=line[1]
            assert len(line)==20, "err, malformed remaining barcode"
            
            umi=line[:umi_len]
            pBC=line[umi_len:]

#unfinished

break_up_u6_count("retina_u6_r1.count","retina_u6_r1")
break_up_u6_count("retina_u6_r2.count","retina_u6_r2")

      1 AAAAAACAGCTCATAT_CACGCAAATTTAAAGGAGCT

1
AAAAAACAGCTCATAT
CACGCAAATTTA
AAGGAGCT
